In [1]:
#
# Import Libraries
#

import os
import sys
import pandas as pd
import numpy as np

utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

from BasicStats import Xstats, anova_calc, kruskal_calc, sign_calc

In [2]:
#
# Set constants
#

Qworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\ALDH4"
Mworking_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\ALDH4"

# X
xq_path = os.path.join(Qworking_path, "WorkingFiles", "Xq_minus_X_norm.tsv")
mq_path = os.path.join(Mworking_path, "WorkingFiles", "Xm_norm.tsv")

# Feature info
q2i_path = os.path.join(Qworking_path, "WorkingFiles", "q2info.tsv")
m2i_path = os.path.join(Mworking_path, "WorkingFiles", "f2info.tsv")

# metadata
mdata_path = os.path.join(r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metadata\ALDH4\WorkingFiles\main_metadata.tsv")

In [3]:
#
# Read data
#

xq = pd.read_csv(xq_path, sep='\t', index_col=0)
xm = pd.read_csv(mq_path, sep='\t', index_col=0)

#q2i = pd.read_csv(q2i_path, sep='\t')
#m2i = pd.read_csv(m2i_path, sep='\t')

mdata = pd.read_csv(mdata_path, sep='\t').set_index('Seqn')

In [5]:
xqs = anova_calc(xq, mdata, factor='Group')
xms = anova_calc(xm, mdata, factor='Group')

In [6]:
xqs = xqs.join(kruskal_calc(xq, mdata, 'Group'))
xms = xms.join(kruskal_calc(xm, mdata, 'Group'))

In [7]:
xqs = xqs.join(sign_calc(xq, mdata, 'Group', 'A12', 'PBS'))
xqs = xqs.join(sign_calc(xq, mdata, 'Group', 'B1-8', 'PBS'))

xms = xms.join(sign_calc(xm, mdata, 'Group', 'A12', 'PBS'))
xms = xms.join(sign_calc(xm, mdata, 'Group', 'B1-8', 'PBS'))

In [8]:
xqs.to_csv('Xq_stats.tsv', sep='\t')
xms.to_csv('Xm_stats.tsv', sep='\t')